In [5]:
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
### Fundamental parameters
e_charge = 1.6022e-19 # [C] electron charge
e_charge_cgs = 4.8e-10  # [statcoul] 
mass_proton = 1.6726e-24 # [g] proton mass 
mass_electron = 9.109e-28  # [g] electron mass
c_cgs         = 3.0e10     # [cm/s]
gamma = 5./3 # adiabatic index
K_Bol = 1.38e-16 # Boltsmann constant in CGS units

In [33]:
### Input parameters for the plasma system in CGS units
### except T_e, T_i are in [eV]

# magnetic field compression
V = 1.3e7; 
B = 300.e4; # just a place holder
T_e = 350.; 
T_i = 350.; 
n_e = 1.0e20; 
# n_e = np.logspace(17, 21, 1000);
space_scale = 0.1;  # just a place holder
A = 64.0; 
Z = 19.0  

In [34]:
m_i = A * mass_proton   # [g]
n_i = n_e/Z             # [cm-3]
rho = n_i*m_i           # [g/cm3]

### Printing input parameters: ### 
print ('')
print ('Parameters:------------------')
print ('')
print ('Flow velocity V           = {:.1e}'.format(V),           '[cm/s]')
print ('B-field                   = {:.1e}'.format(B),           '[G]')
print ('Electron temperature T_e  = {:.1f}'.format(T_e),         '   [eV]')
print ('Ion temperature T_i       = {:.1f}'.format(T_i),         '   [eV]')
print ('Electron density n_e      = {:.1e}'.format(n_e),         '[cm-3]')
print ('Ion density n_i           = {:.1e}'.format(n_i),         '[cm-3]')   
print ('Density rho               = {:.1e}'.format(rho),         '[g/cm-3]')
print ('Charge state Z            = {:.1f}'.format(Z))
print ('Mass number A             = {:.1f}'.format(A))
print ('Spacial scale             = {:.1e}'.format(space_scale), '[cm]')

print ('')


Parameters:------------------

Flow velocity V           = 1.3e+07 [cm/s]
B-field                   = 3.0e+06 [G]
Electron temperature T_e  = 350.0    [eV]
Ion temperature T_i       = 350.0    [eV]
Electron density n_e      = 1.0e+20 [cm-3]
Ion density n_i           = 5.3e+18 [cm-3]
Density rho               = 5.6e-04 [g/cm-3]
Charge state Z            = 19.0
Mass number A             = 64.0
Spacial scale             = 1.0e-01 [cm]



In [35]:
### Sound velocity and Alfven velocity
c_sound = np.sqrt(5.0*(n_i*T_i*K_Bol*1.16e4+n_e*T_e*K_Bol*1.16e4)/(3.0*rho)) # P_ther*gamma = rho*V_s^2 (1 eV = 1.16e4 K)
print ('Sound velocity            = {:.1e}'.format(c_sound),  '[cm/s]')
c_alfven = B/(4*np.pi*n_i*m_i)**(1./2) # Formula from NRL p.29
print ('Alfven velocity           = {:.1e}'.format(c_alfven), '[cm/s]')
c_magnetosonic = np.sqrt(c_sound**2 + c_alfven**2)
print ('c_magnetosonic velocity   = {:.1e}'.format(c_magnetosonic), '[cm/s]')

print ('')

Sound velocity            = 1.3e+07 [cm/s]
Alfven velocity           = 3.6e+07 [cm/s]
c_magnetosonic velocity   = 3.8e+07 [cm/s]



In [27]:
### ion inertial length
di = 2.28e7 * Z**(-1) * (A/n_i)**(0.5)  #[cm]
print ('Ion inertial length di    = {:.1f}'.format(di*1e4),     '[um]')

### electron inertial length
de = 5.31e5 * (1/n_e)**(0.5)  #[cm]
print ('Electron inertial length de    = {:.1f}'.format(de*1e4),     '[um]')
print(" ")

Ion inertial length di    = 41.8 [um]
Electron inertial length de    = 0.5 [um]
 


In [28]:
## when streaming/drifting, more care should be taken.

## first, go to the background plasma reference frame and get the streaming velocity
v_rel = V * 2
print('relative velocity v_rel = {:.2e} [cm/s]'.format(v_rel))
print('')
## then, check if the streaming particle is slow or fast

## for ion-ion
x_ii = m_i * v_rel**2 / (2*T_i*K_Bol*1.16e4)
print('for ion-ion, x_ii = {:.2e}'.format(x_ii))
x_ie = mass_electron * v_rel**2 / (2*T_e*K_Bol*1.16e4)
print('for ion-eon, x_ie = {:.2e}'.format(x_ie))
x_ei = m_i * v_rel**2 / (2*T_i*K_Bol*1.16e4)
print('for eon-ion, x_ei = {:.2e}'.format(x_ei))
x_ee = mass_electron * v_rel**2 / (2*T_e*K_Bol*1.16e4)
print('for eon-eon, x_ee = {:.2e}'.format(x_ee))
print(' ')

relative velocity v_rel = 2.60e+07 [cm/s]

for ion-ion, x_ii = 6.46e+01
for ion-eon, x_ie = 5.50e-04
for eon-ion, x_ei = 6.46e+01
for eon-eon, x_ee = 5.50e-04
 


In [29]:
## next, calculate oen of the collisional rates (the slowing down one) accordingly

ekin_i = 0.5*(m_i*1e-3)*(V*1e-2)**2 / e_charge # [eV]
print('ekin_i = {:.2e} eV'.format(ekin_i))

ekin_e = 0.5*(mass_electron*1e-3)*(V*1e-2)**2 / e_charge # [eV]
print('ekin_e = {:.2e} eV'.format(ekin_e))
print(' ')

# for ion-ion
cou_log_ii = 43. - np.log(Z*Z*(A+A)/(A*A*v_rel**2/c_cgs**2) * (n_e/T_e)**(0.5))
print('Coul. Log. of ion-ion = ', cou_log_ii)
print(' ')
if x_ii < 1:    
    f_ii = 6.8e-8 * A**(-0.5) * 2 * T_i**(-1.5) * n_i * Z**4 * cou_log_ii
else:
    f_ii = 9.0e-8 * (1./A + 1./A) * A**(0.5) / ekin_i**(1.5) * n_i * Z**4 * cou_log_ii
print('collision frequency f_ii   = {:.2e} s^-1'.format(f_ii))
print('collision time      tau_ii = {:.1e} ns'.format(1./f_ii*1e9))

ekin_i = 5.65e+03 eV
ekin_e = 4.80e-02 eV
 
Coul. Log. of ion-ion =  6.3782613465225
 
collision frequency f_ii   = 2.32e+11 s^-1
collision time      tau_ii = 4.3e-03 ns


In [30]:
# for ion-eon
if T_i * mass_electron / m_i < T_e < 10. * Z**2:
    cou_log_ie = 23. - np.log(n_e**0.5 * Z * T_e**(-1.5))
elif 10. * Z**2 < T_e:
    cou_log_ie = 24. - np.log(n_e**0.5 * T_e**(-1))
elif T_e < T_i * mass_electron / m_i:
    cou_log_ie = 16. - np.log(n_i**0.5 * T_i**(-1.5) * Z**2 * A)
print('Coul. Log. of ion-eon = ', cou_log_ie)
print(' ')

if x_ie < 1:
    f_ie = 1.6e-9 * A**(-1) * T_e**(-1.5) * n_e * Z**2 * cou_log_ie
else:
    f_ie = 1.7e-4 * A**0.5 * ekin_i**(-1.5) * n_e * Z**2 * cou_log_ie
print('collision frequency f_ie   = {:.2e} s^-1'.format(f_ie))
print('collision time      tau_ie = {:.1f} ns'.format(1./f_ie*1e9))

Coul. Log. of ion-eon =  5.816609822618293
 
collision frequency f_ie   = 8.02e+08 s^-1
collision time      tau_ie = 1.2 ns


In [31]:
# for eon-ion
cou_log_ei = cou_log_ie
print('Coul. Log. of eon-ion = ', cou_log_ei)
print(' ')
if x_ei < 1:
    f_ei = 0.23 * A**1.5 / T_i**1.5 * n_i * Z**2 * cou_log_ei
else:
    f_ei = 3.9e-6 / ekin_e**1.5 * n_i * Z**2 * cou_log_ei
print('collision frequency f_ei   = {:.2e} s^-1'.format(f_ei))
print('collision time      tau_ei = {:.1e} ns'.format(1./f_ei*1e9))

Coul. Log. of eon-ion =  5.816609822618293
 
collision frequency f_ei   = 4.09e+18 s^-1
collision time      tau_ei = 2.4e-10 ns


In [32]:
# for eon-eon
cou_log_ee = 23.5 - np.log(n_e**0.5 / T_e**1.25) - (1e-5 + (np.log(T_e)-2.)**2 / 16.)**0.5
print('Coul. Log. of ion-eon = ', cou_log_ee)
print(' ')

if x_ee < 1:
    f_ee = 5.8e-6 / T_e**1.5 * n_e * cou_log_ee
else:
    f_ee = 7.7e-6 / ekin_e**1.5 * n_e * cou_log_ee
print('collision frequency f_ee   = {:.2e} s^-1'.format(f_ee))
print('collision time      tau_ee = {:.1e} ns'.format(1./f_ee*1e9))   

Coul. Log. of ion-eon =  6.832077040433935
 
collision frequency f_ee   = 6.05e+11 s^-1
collision time      tau_ee = 1.7e-03 ns
